In [2]:
import os
from PIL import Image
import numpy as np
import pickle
import cv2
from mtcnn.mtcnn import MTCNN
from calibration import Calibration
import dlib
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
from keras.models import load_model
model = load_model('./trained_models/model.h5')
model_eye = load_model('./trained_models/eye_model.h5')
calibration = Calibration()
_predictor = dlib.shape_predictor("trained_models/shape_predictor_68_face_landmarks.dat")
detector = MTCNN()

W0507 13:49:49.596323 72692 deprecation_wrapper.py:119] From C:\Users\asus\Anaconda3\envs\ML_GPU\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0507 13:49:49.613246 72692 deprecation_wrapper.py:119] From C:\Users\asus\Anaconda3\envs\ML_GPU\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0507 13:49:49.643166 72692 deprecation_wrapper.py:119] From C:\Users\asus\Anaconda3\envs\ML_GPU\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0507 13:49:49.645161 72692 deprecation_wrapper.py:119] From C:\Users\asus\Anaconda3\envs\ML_GPU\lib\site-packages\keras\backend\tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0507 13:49:49.646158 72692 deprec

In [4]:
def get_pixele_test(img_array, required_size=(60, 60)):
    image = Image.fromarray(img_array)
    image = image.resize(required_size)
    face_array = np.asarray(image)/255
    return face_array

def preprocess_img(img):
    img = Image.fromarray(img)
    img = img.resize((30,30))
    img = np.array(img)
    img = img/255
    img = img.reshape(30, 30, 1)
    img = np.expand_dims(img,axis=0)
    return img

In [5]:
def prediction_eyes(faces,frame_g):
    bounding_box = faces[i]['box']
    x1, y1, width, height = bounding_box
#         face = [(x1,y1),(x1+width,y1+height)]
    face = dlib.rectangle(x1,y1,x1+width,y1+height)
    landmarks = _predictor(frame, face)
    left_eye = []
    right_eye = []
    left_eye.append([landmarks.part(36).x,landmarks.part(36).y])
    left_eye.append([landmarks.part(37).x,landmarks.part(37).y])
    left_eye.append([landmarks.part(38).x,landmarks.part(38).y])
    left_eye.append([landmarks.part(39).x,landmarks.part(39).y])
    left_eye.append([landmarks.part(40).x,landmarks.part(40).y])
    left_eye.append([landmarks.part(41).x,landmarks.part(41).y])

    right_eye.append([landmarks.part(42).x,landmarks.part(42).y])
    right_eye.append([landmarks.part(43).x,landmarks.part(43).y])
    right_eye.append([landmarks.part(44).x,landmarks.part(44).y])
    right_eye.append([landmarks.part(45).x,landmarks.part(45).y])
    right_eye.append([landmarks.part(46).x,landmarks.part(46).y])
    right_eye.append([landmarks.part(47).x,landmarks.part(47).y])

    left_eye = np.array(left_eye)
    right_eye = np.array(right_eye)

    rect_left = cv2.boundingRect(left_eye)
    x,y,w,h = rect_left
    txt_left = (x,y)
    croped_left = frame_g[y:y+h, x:x+w].copy()

    left_eye = left_eye - left_eye.min(axis=0)
    cv2.polylines(croped_left, np.array([left_eye], int), True, (0,0,0), 2)
    mask_left = np.zeros(croped_left.shape[:2], np.uint8)
    cv2.drawContours(mask_left, [left_eye], -1, (255, 0, 0), -1, cv2.LINE_AA)
    dst_left = cv2.bitwise_and(croped_left, croped_left, mask=mask_left)
#         cv2.imshow("left",dst_left)

    rect_right = cv2.boundingRect(right_eye)
    x,y,w,h = rect_right
    txt_right = (x,y)
    croped_right = frame_g[y:y+h, x:x+w].copy()
    right_eye = right_eye - right_eye.min(axis=0)
    cv2.polylines(croped_right, np.array([right_eye], int), True, (0,0,0), 2)
    mask_right = np.zeros(croped_right.shape[:2], np.uint8)
    cv2.drawContours(mask_right, [right_eye], -1, (255, 0, 0), -1, cv2.LINE_AA)
    dst_right = cv2.bitwise_and(croped_right, croped_right, mask=mask_right)
#         cv2.imshow("right",dst_right)
    left_eye_img = preprocess_img(dst_left)
    right_eye_img = preprocess_img(dst_right)

    left_pred = model_eye.predict(left_eye_img)
    right_pred = model_eye.predict(right_eye_img)
    pred_eye = []
    pred_eye.append(left_pred)
    pred_eye.append(right_pred)
    return pred_eye


In [6]:
# cam = cv2.VideoCapture(0)
# font = cv2.FONT_HERSHEY_SIMPLEX 
 
# fontScale = 1
# color = (255, 0, 0) 
# thickness = 2

# while(True):
#     ret,frame = cam.read()
#     key_pressed = cv2.waitKey(1)&0xFF
#     if ret==False:
#         print("Something went wrong")
#         continue
#     if key_pressed == ord('q'):
#         break
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# #     frame = cv2.flip(frame, 1)
#     faces = detector.detect_faces(frame)
#     for i in range(len(faces)):
#         bb = faces[i]['box']
#         x, y, w, h = bb
#         f_img = frame[y:y+h,x:x+w]
#         face = get_pixele_test(f_img)
#         face = np.expand_dims(face, axis=0)
#         pred = model.predict(face)
        
#         if pred>0.5:
#             frame = cv2.rectangle(frame, (int(x),int(y)), (int(x+w),int(y+h)), (0, 255, 0), thickness)
#             pred_eyes = prediction_eyes(faces,frame_g)
# #             print(pred_eyes)
#             if pred_eyes[0]>0.5 or pred_eyes[1]>0.5:
#                 cv2.putText(frame,"cheating_eyes",(50,50),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 2, cv2.LINE_AA)
#             else:
#                 cv2.putText(frame,"ok",(50,50),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 1, cv2.LINE_AA)
        
#         else:
#             frame = cv2.rectangle(frame, (int(x),int(y)), (int(x+w),int(y+h)), (255, 0, 0), thickness)
            
#             pred_eyes = prediction_eyes(faces,frame_g)
# #             print(pred_eyes)
#             if pred_eyes[0]>=0.9 and pred_eyes[1]>=0.9:
#                 cv2.putText(frame,"cheating_eyes",(50,50),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 1, cv2.LINE_AA)
#             else:
#                 cv2.putText(frame,"Ok",(50,50),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 1, cv2.LINE_AA)
    
    
    
#     cv2.imshow('yo', frame)
# cv2.waitKey()
# cv2.destroyAllWindows()
    

In [8]:
cams = []
path = "./final_dataset/test"
for root,dirs,files in os.walk(path):
    for file in files:
        cams.append(cv2.VideoCapture(path+'/'+file))
font = cv2.FONT_HERSHEY_SIMPLEX 
 
fontScale = 1
color = (255, 0, 0) 
thickness = 2

while(True):
    for n,cam in enumerate(cams[:2]):   
        ret,frame = cam.read()
        frame = Image.fromarray(frame)
        frame = frame.resize((640,480))
        frame = np.array(frame)
        
        key_pressed = cv2.waitKey(1)&0xFF
    #     frame = final_frame
        if ret==False:
            print("Something went wrong")
            continue
        if key_pressed == ord('q'):
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #     frame = cv2.flip(frame, 1)
        faces = detector.detect_faces(frame)
    #     print(len(faces))
        for i in range(len(faces)):
            bb = faces[i]['box']
            x, y, w, h = bb
            f_img = frame[y:y+h,x:x+w]
            face = get_pixele_test(f_img)
            face = np.expand_dims(face, axis=0)
            pred = model.predict(face)
            if pred>0.5:
                frame = cv2.rectangle(frame, (int(x),int(y)), (int(x+w),int(y+h)), (0, 255, 0), thickness)
            else:
                frame = cv2.rectangle(frame, (int(x),int(y)), (int(x+w),int(y+h)), (255, 0, 0), thickness)
                pred_eyes = prediction_eyes(faces,frame_g)
    #             print(pred_eyes)
                if pred_eyes[0]>=0.5 and pred_eyes[1]>=0.5:
                    cv2.putText(frame,"cheating_eyes",(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 1, cv2.LINE_AA)
                else:
                    cv2.putText(frame,"Ok",(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 1, cv2.LINE_AA)


        if n==0:
            final_frame = frame
        else:
            final_frame = np.hstack((final_frame,frame))
    cv2.imshow('yo', final_frame)
cv2.waitKey()
cv2.destroyAllWindows()
    

KeyboardInterrupt: 